# Discover and Run Standard Infographics

This is a simple notebook demonstrating how to discover the standard Infographics included with Business Analyst, and
export Infographics to local files.

### Imports and Setup

We start by importing needed Python assets, ensuring all the needed paths exist and connect to an ArcGIS instance. In this example, I am using a saved profile to connect to ArcGIS Online.

In [2]:
import importlib.util
import json
from pathlib import Path
import sys

from arcgis.gis import GIS
from arcgis.geometry import Geometry
from arcgis.geoenrichment import create_report

# get the path to the parent paroject directory
dir_prj = Path.cwd().parent

# paths for where to save output infograhpics and where to find the supporting Python package
dir_reports = dir_prj / 'reports'
dir_src = dir_prj / 'src'

# make sure can import the local project
if importlib.util.find_spec('infographics') is None:
    sys.path.append(str(dir_prj/'src'))

if importlib.util.find_spec('infographics') is None:
    raise EnvironmentError('Cannot access "infographics" Python package. Please ensure it is available.')

# import the local Python project helper function for discovering available infographics
from infographics import get_standard_infographics

# make sure the place to save the reports exists
dir_reports.mkdir(parents=True, exist_ok=True)

# create arcgis instance using a saved profile
# REF: https://developers.arcgis.com/python/latest/guide/working-with-different-authentication-schemes/#protecting-your-credentials
gis = GIS(profile='bateam')

## See What Infographics Are Available

Discovering Item ID's for the standard Infographics available through your GIS (either Business Analyst Enterprise or Business Analyst Web) can be accomplished using a small helper function I created `get_standard_infographics`. This function queries the resources available through the provided `GIS` object instance, and returns a Pandas Dataframe of available standard Infographics. The most useful column is `itemID`, the code you will use to create Infographics below.

In [8]:
get_standard_infographics('US', gis=gis)

,reportID,title,itemID,formats,dataVintage,countries,hierarchy,category
0,2020-census-summary,2020 Census Summary,fbe85991e2c8424b9643e105e56b1987,"[pdf, html, xlsx]","2020, 2000, 2010",US,esri2024,standard
1,at-risk-population,At Risk Population,c03642993aef454cacf827569a226ddb,"[pdf, html, xlsx]","2024, 2029, 2018-2022",US,esri2024,standard
2,childhood-and-female-equity,Childhood and Female Equity,b82d209c6e58446c9882654d7526d3e3,"[pdf, html, xlsx]","2024, 2018-2022",US,esri2024,standard
3,community-change-snapshot,Community Change Snapshot,142f5fc8e3fd43b0b873afc0a1f652da,"[pdf, html, xlsx]","2010, 2020, 2024, 2029, 2000, 2018-2022",US,esri2024,standard
4,community-summary,Community Summary,6bdd88c878994888bf8c289acd35ca60,"[pdf, html, xlsx]","2024, 2018-2022",US,esri2024,standard
5,commute-profile,Commute Profile,f3b68130833e437b91bd7fa5f944d458,"[pdf, html, xlsx]",2018-2022,US,esri2024,standard
6,demographic-profile,Demographic Profile,958cf782fd204fcabf204918e73c394a,"[pdf, html, xlsx]","2024, 2029",US,esri2024,standard
7,demographic-summary,Demographic Summary,93ce5ce3013a4368ae8060a37cd438d8,"[pdf, html, xlsx]","2024, 2029",US,esri2024,standard
8,dominant-tapestry-profile,Dominant Tapestry Profile,7d2dea41ab72485faa4022b1eb3690c4,"[pdf, html, xlsx]","2024, 2029, 2018-2022",US,esri2024,standard
9,eating-places,Eating Places,e4bc006c86e6437b83783d9d8e7443f6,"[pdf, html, xlsx]","2024, 2029",US,esri2024,standard


## Get Geographies

Get a few US Census tracts to work with from the Living Atlas to use for creating Infographics. This is where you should get your own unique identifiers and geometries to work with. Rather than create a Notebook with an example you will not be able to run, I am using this data since you will be able to access it and run this example locally if you want to experiment.

In [3]:
# US Census Tracts from the Living Atlas
itm_id = '20f5d275113e4066bf311236d9dcc3d4'

# a few Tracts along the Washington side of the Columbia River Gorge
fips_lst = [
    '53039950301',
    '53039950302',
    '53059950400'
]

In [4]:
# create SQL to retrieve just three US Census tracts from the Living Atlas
fips_str = ",".join([f"'{fips}'" for fips in fips_lst])

sql_clause = f'FIPS IN ({fips_str})'

sql_clause

"FIPS IN ('53039950301','53039950302','53059950400')"

In [5]:
# create a layer and retrieve the three features as a Pandas Spatially Enabled Data Frame
lyr = gis.content.get(itm_id).layers[0]

sedf = lyr.query(sql_clause, out_fields='FIPS', out_sr=4326).sdf

sedf

,OBJECTID,FIPS,SHAPE
0,81037,53039950302,"{""rings"": [[[-121.266483346, 45.947432635], [-..."
1,81040,53039950301,"{""rings"": [[[-121.483686332, 45.7125475990001]..."
2,81360,53059950400,"{""rings"": [[[-121.627693389, 45.8690515930001]..."


## Create and Save Infographics

This is the interesting part, how to get the Infographics saved locally. You can either use the `create_reports` function directly, or take advantage of a small helper function `create_infographics`, I created to make life easier by wraping up geometry formatting and a little error catching.

### Directly Use `create_report`

Using the `create_report` function is not too difficult provided you ensure the geometries are correctly formatted first.

In [7]:
# infographic id discovered above and output file name pattern root
infographic_item_id = '9ea375b7623c4142b0fcb03f2936640c'
output_file_name_root = 'employment_overview'

# iterate the features to save a discrete infographc for each
for _, fips, geom in sedf[['FIPS', 'SHAPE']].itertuples():
    
    # create the name for the the infographic
    out_nm = f'infographic_{output_file_name_root}_{fips}.htm'

    # format geometries as list of dicts so create_report can handle
    in_geom = [{"geometry": json.loads(geom.JSON)}]

    # create the infographic
    res = create_report(
        study_areas=in_geom,
        report=infographic_item_id,
        export_format='html',
        out_folder=dir_reports,
        out_name=out_nm,
        gis=gis,
    )

    # report success
    print(f'Saved infographic {res}')

Saved infographic D:\projects\python-infographics\reports\infographic_employment_overview_53039950302.htm
Saved infographic D:\projects\python-infographics\reports\infographic_employment_overview_53039950301.htm
Saved infographic D:\projects\python-infographics\reports\infographic_employment_overview_53059950400.htm


### Use Helper Function `create_infographics`

If you want the geomtery formatting taken care of for you, I wrapped the `create_report` function with a little error catching to make life a little easier.

In [6]:
# import the helper function from the local Python package
# NOTE: best practice is to do it at the top with other imports, but doing here to emphasize where it is coming from
from infographics import create_infographic

# infographic id discovered above and output file name pattern root
infographic_item_id = '9ea375b7623c4142b0fcb03f2936640c'
output_file_name_root = 'employment_overview'

# iterate the features to save a discrete infographc for each
for _, fips, geom in sedf[['FIPS', 'SHAPE']].itertuples():
    
    # create path to save the the infographic
    out_nm = dir_reports / f'infographic_{output_file_name_root}_{fips}.htm'

    # create the infographic
    res = create_infographic(
        study_areas=geom,
        gis=gis,
        infographic_id=infographic_item_id,
        out_path=out_nm,
        export_format='html'
    )

    # report success
    print(f'Saved infographic {res}')

Saved infographic D:\projects\python-infographics\reports\infographic_employment_overview_53039950302.htm
Saved infographic D:\projects\python-infographics\reports\infographic_employment_overview_53039950301.htm
Saved infographic D:\projects\python-infographics\reports\infographic_employment_overview_53059950400.htm
